# Bert on 40-10-50 split with 2e-5 learning rate

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from wordcloud import WordCloud  
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# these libraries are used for the text cleaning
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#these libraries are building the model
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler

# seed for reproducibility
seed = 1111

In [ ]:
# load the dataframes
train_data = pd.read_csv("train_review_data.csv",usecols=["review", "label"])
test_data = pd.read_csv("test_review_data.csv", usecols=["review", "label"])
full_data = pd.read_csv("full_review_data.csv", usecols=["review", "label"])

In [ ]:
X_train = list(train_data['review'])
X_test = list(test_data['review'])
Y_train = list(train_data['label'])
Y_test = list(test_data['label'])

#splitting the train set into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train, test_size=0.2, random_state=seed)

In [ ]:
# Construct a BERT tokenizer, reutrns a list of input IDs with the appropriate special tokens.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing function
def tokenize_data(texts, tokenizer, max_len=512):
    return tokenizer(
        texts,
        max_length=max_len,
        padding="max_length",  # Pads to max_len
        truncation=True,      # Truncates to max_len
        return_tensors="pt"   # Returns PyTorch tensors
    )

# Tokenize datasets
train_encodings = tokenize_data(X_train, tokenizer)
val_encodings = tokenize_data(X_val, tokenizer)
test_encodings = tokenize_data(X_test, tokenizer)

In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = IMDBDataset(train_encodings, Y_train)
val_dataset = IMDBDataset(val_encodings, Y_val)
test_dataset = IMDBDataset(test_encodings, Y_test)

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()

# Added seed for reproducibility
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
epochs = 3

# reference: https://stackoverflow.com/questions/60120043/optimizer-and-scheduler-for-bert-fine-tuning
num_training_steps = len(train_loader) * epochs
scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

def train_epoch(model, dataloader, optimizer, device, scheduler):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(dataloader), correct / total

In [ ]:
def evaluate_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []  # To store predictions
    all_labels = []  # To store ground truth labels

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            # Get predicted class indices
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Store predictions and labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Return predictions and labels along with loss and accuracy
    return total_loss / len(dataloader), correct / total, all_preds, all_labels

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

all_test_preds = []
all_test_labels = []

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device, scheduler)
    val_loss, val_acc, val_preds, val_labels = evaluate_epoch(model, val_loader, device)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}")

# Evaluate on the test set and retrieve predictions and labels
test_loss, test_acc, test_preds, test_labels = evaluate_epoch(model, test_loader, device)
print()
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy', marker='o')
plt.plot(range(1, epochs + 1), val_accuracies, label='Validation Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()

# Show plots
plt.tight_layout()
plt.show()

In [ ]:
# Generate confusion matrix for the test set
cm = confusion_matrix(test_labels, test_preds)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix for Test Data")
plt.show()

# BERT Embeddings and Clustering

In [ ]:
def extract_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []
    labels_list = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)  # to keep track of labels

            # forward pass
            outputs = model.bert(input_ids, attention_mask=attention_mask)

            # CLS token embedding (the first token) (access the hidden states because they contain the embeddings that capture the meaning of the text)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # shape: (batch_size, hidden_size)

            embeddings.append(cls_embeddings.cpu())  # store embeddings on the CPU since it is bigger
            labels_list.append(labels.cpu())  # store labels

    embeddings = torch.cat(embeddings, dim=0)  # combine all batches
    labels_list = torch.cat(labels_list, dim=0)  # combine all labels
    return embeddings, labels_list

In [ ]:
# extract embeddings for the training, validation, and test set
train_embeddings, train_labels = extract_embeddings(model, train_loader, device)
val_embeddings, val_labels = extract_embeddings(model, val_loader, device)
test_embeddings, test_labels = extract_embeddings(model, test_loader, device)

In [ ]:
all_embeddings = torch.cat([train_embeddings, val_embeddings, test_embeddings], dim=0)

In [ ]:
all_labels = torch.cat([train_labels, val_labels, test_labels], dim=0)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# finding the optimal n value using the elbow method
inertia = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X) 
    inertia.append(kmeans.inertia_)

plt.plot(k_range, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# converting to numpy
all_embeddings_np = all_embeddings.numpy()

# K-means
num_clusters = 2  # can choose clusters to be anything
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(all_embeddings_np)

# now cluster_labels contains the cluster assignment for each embedding

In [ ]:
from sklearn.manifold import TSNE

# reduce dimensions to 2D
tsne = TSNE(n_components=2, random_state=42)
reduced_embeddings = tsne.fit_transform(all_embeddings_np)

In [ ]:
import matplotlib.pyplot as plt

# displaying the scatter plot of the reduced embeddings
plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    reduced_embeddings[:, 0],  # X-axis
    reduced_embeddings[:, 1],  # Y-axis
    c=cluster_labels,          
    cmap='viridis',            
    alpha=0.7                  
)

plt.colorbar(scatter)
plt.title('K-means Clustering of BERT Embeddings')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

In [ ]:
# try to see what kinda of reviews are in each cluster
data = pd.DataFrame({
    'cluster': cluster_labels,
    'label': all_labels
})

# group by cluster and count labels
counts = data.groupby(['cluster', 'label']).size().unstack(fill_value=0)

print("Counts of labels in each cluster:")
print(counts)

In [ ]:
from collections import defaultdict

texts = X_train + X_test + X_val

# group texts by their cluster label
cluster_texts = defaultdict(list)
for text, cluster in zip(texts, cluster_labels):
    cluster_texts[cluster].append(text)

cluster_combined_texts = {cluster: " ".join(text_list) for cluster, text_list in cluster_texts.items()}

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

for cluster, text in cluster_combined_texts.items():
    wordcloud = WordCloud(
        width=800, 
        height=400, 
        background_color='white'
    ).generate(text)
    
    # plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Word Cloud for Cluster {cluster}")
    plt.show()